In [ ]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import os

In [ ]:
os.getcwd()

In [ ]:
basepath = 'C:\\Users\\Sarah\\Nexus365\\Tom Russell - ghana-oxford-infrastructure-adaptation-2020'

In [ ]:
ghana = gpd.read_file(os.path.join(basepath, 'incoming\\Ghana.shp'))
ghana = ghana.to_crs("EPSG:32630")

# Energy

In [ ]:
energypath = os.path.join(basepath,'incoming/energy')

## Energy - Transmission Lines

In [ ]:
etransm_path = os.path.join(energypath, 'Energy Transmission Network/ID 4 World Bank Catalog - C.Aderne/ghanafinal.geojson')
GHA_etransm = gpd.read_file(etransm_path)

In [ ]:
GHA_etransm = GHA_etransm.to_crs("EPSG:32630")

In [ ]:
GHA_etransm = gpd.overlay(GHA_etransm, ghana, how='intersection')

In [ ]:
GHA_etransm=GHA_etransm[['admin0Pcod','voltage_kV','year','status','length_km','geometry']]

In [ ]:
GHA_etransm=GHA_etransm.reset_index()

In [ ]:
GHA_etransm=GHA_etransm.rename(columns = {
    'index':'ID', 'admin0Pcod':'NAME', 'voltage_kV':'VOLTAGE_kV','year':'YEAR','status':'STATUS','length_km':'LENGTH_km'
})

In [ ]:
GHA_etransm.head()

In [ ]:
GHA_etransm.to_file(os.path.join(
basepath,'data','infrastructure','GHA_transmissionlines.gpkg'), layer='transmissionlines', driver="GPKG")

## Electricity Grid - ID82 gridfinder

In [ ]:
grid_path = os.path.join(energypath, 'Energy Transmission Network/ID82 gridfinder/grid.gpkg')
GHA_grid = gpd.read_file(grid_path)

In [ ]:
GHA_grid = GHA_grid.to_crs("EPSG:32630")

In [ ]:
GHA_grid = gpd.overlay(GHA_grid, ghana, how='intersection')

In [ ]:
GHA_grid = GHA_grid[['admin0Pcod','geometry']]

In [ ]:
GHA_grid = GHA_grid.reset_index()

In [ ]:
GHA_grid = GHA_grid.rename(columns = {'index':'ID','admin0Pcod':'NAME'})

In [ ]:
GHA_grid.head()

In [ ]:
GHA_grid.to_file(os.path.join(
basepath,'data','infrastructure','GHA_grid.gpkg'), layer='grid', driver="GPKG")

## Energy - Substations

### Substations - OSM data

Subset 1

In [ ]:
substation1_path = os.path.join(energypath, 'Energy Transmission Network/OSM substations/OSM_substations1.shp')
GHA_subst1 = gpd.read_file(substation1_path)

In [ ]:
GHA_subst1 = GHA_subst1.to_crs("EPSG:32630")

In [ ]:
GHA_subst1 = GHA_subst1[['name','substation','voltage','geometry']]
GHA_subst1.reset_index()
GHA_subst1 = GHA_subst1.rename(columns = {'index':'ID','name':'NAME','substation':'TYPE','voltage':'VOLTAGE'})

In [ ]:
GHA_subst1.head(2)

Subset 2

In [ ]:
substation2_path = os.path.join(energypath, 'Energy Transmission Network/OSM substations/OSM_substations2.shp')
GHA_subst2 = gpd.read_file(substation2_path)

In [ ]:
GHA_subst2 = GHA_subst2.to_crs("EPSG:32630")

In [ ]:
GHA_subst2 = GHA_subst2[['name','substation','voltage','operator','geometry']]
GHA_subst2.reset_index()
GHA_subst2 = GHA_subst2.rename(columns = {'index':'ID','name':'NAME','substation':'TYPE','voltage':'VOLTAGE','operator':'OPERATOR'})

In [ ]:
GHA_subst2.head(2)

Subset 3

In [ ]:
substation3_path = os.path.join(energypath, 'Energy Transmission Network/OSM substations/OSM_substations3.shp')
GHA_subst3 = gpd.read_file(substation3_path)

In [ ]:
GHA_subst3 = GHA_subst3.to_crs("EPSG:32630")

In [ ]:
GHA_subst3 = GHA_subst3[['name','substation','operator','geometry']]
GHA_subst3.reset_index()
GHA_subst3 = GHA_subst3.rename(columns = {'index':'ID','name':'NAME','substation':'TYPE','operator':'OPERATOR'})

In [ ]:
GHA_subst3.head(2)

### Concatenate three dataframes

In [ ]:
GHA_subst_concat = pd.concat([GHA_subst2, GHA_subst3, GHA_subst1], sort=False)
GHA_subst_concat = GHA_subst_concat.reset_index()
GHA_subst_concat = GHA_subst_concat.rename(columns = {'index':'ID'})

In [ ]:
GHA_subst_concat.head()

In [ ]:
GHA_subst_concat.to_file(os.path.join(
basepath,'data','infrastructure','GHA_substations.gpkg'), layer='substations', driver="GPKG")

## Energy - Power Plants

There are two dataframes with powerplants, one from the World Bank Catalog and one from ECOVREX

### Dataframe Powerplants World Bank Catalog:

In [ ]:
powerWB_path = os.path.join(energypath, 'Power plants & stations/ID 1 - World Bank Catalog (WEPP2006)/GHA_PowerPlants.shp')
GHA_powerplWB = gpd.read_file(powerWB_path)

In [ ]:
GHA_powerplWB = GHA_powerplWB.set_crs("EPSG:32630")

In [ ]:
GHA_powerplWB = GHA_powerplWB[['PLANT','GEN_TYPE','STATUS','SUM_MW','geometry',]]

GHA_powerplWB=GHA_powerplWB.rename(columns = {
        'PLANT':'NAME', 'GEN_TYPE':'TYPE','SUM_MW':'CAPACITY_MW'
    })

In [ ]:
GHA_powerplWB.head()

### Dataframe from ECOVREX:

In [ ]:
powerECOV_path = os.path.join(energypath, 'Power plants & stations/ID67-ecreee_energy_generators/ecreee_energy_generators.shp')
GHA_powerplECOV = gpd.read_file(powerECOV_path)

In [ ]:
GHA_powerplECOV = GHA_powerplECOV[GHA_powerplECOV.geometry.notnull()]
GHA_powerplECOV = GHA_powerplECOV.to_crs("EPSG:32630")

In [ ]:
GHA_powerplECOV = GHA_powerplECOV[(GHA_powerplECOV.country == "Ghana")]

In [ ]:
GHA_powerplECOV = GHA_powerplECOV[['title','eg_type','current_st','current_ca','geometry']]
GHA_powerplECOV = GHA_powerplECOV.rename(columns = {'title':'NAME','eg_type':'TYPE','current_st':'STATUS','current_ca':'CAPACITY_MW'})

Deleted columns: nid, eg_type_id, initiative, ree_cat, Size, Longitude & Latitude

In [ ]:
GHA_powerplECOV.head()

### Concatenate both data frames

just removed , keys=['PP_WB','PP_ECOV'] to fix Index

In [ ]:
GHA_powerpl_concat = pd.concat([GHA_powerplWB, GHA_powerplECOV], sort=False)
GHA_powerpl_concat = GHA_powerpl_concat.reset_index()
GHA_powerpl_concat = GHA_powerpl_concat.rename(columns = {'index':'ID'})

In [ ]:
GHA_powerpl_concat.head(20)

In [ ]:
GHA_powerpl_concat.to_file(os.path.join(
basepath,'data','infrastructure','GHA_powerplant.gpkg'), layer='powerplants', driver="GPKG")

In [ ]:
GHA_powerpl_opr1 = GHA_powerpl_concat[(GHA_powerpl_concat.STATUS == "OPR")]
GHA_powerpl_opr2 = GHA_powerpl_concat[(GHA_powerpl_concat.STATUS == "Operatinal")]

GHA_powerpl_opr = pd.concat([GHA_powerpl_opr1, GHA_powerpl_opr2], sort=False)

In [ ]:
GHA_powerpl_opr = GHA_powerpl_opr[['NAME','TYPE','CAPACITY_MW','geometry']]
GHA_powerpl_opr = GHA_powerpl_opr.reset_index()
GHA_powerpl_opr = GHA_powerpl_opr.rename(columns ={'index':'ID'})

In [ ]:
GHA_powerpl_opr.head()

In [ ]:
GHA_powerpl_opr.to_file(os.path.join(
basepath,'data','infrastructure','Points','GHA_powerplant_opr.gpkg'), layer='powerplants_opr', driver="GPKG")